In [9]:
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 

class TwitterClient(object): 
	''' 
	Generic Twitter Class for sentiment analysis. 
	'''
	def __init__(self): 
		''' 
		Class constructor or initialization method. 
		'''
		# keys and tokens from the Twitter Dev Console 
		consumer_key = 'vtMN7CD9BwCniLHMV11qIkCaw'
		consumer_secret = 'iTdMHcFaZY9XUWj2TbS70sYMKvvkdeYiPPhHXLyKKdfzVC2Xs1'
		access_token = '1269149138555305984-UX70uSINlo4a3zNUFB8HDC1RJEWFh2'
		access_token_secret = 'QK2jYD87eR1Rdue3sWEIO55g5vNKgmVLYUlgtsNnWIjuc'

		# attempt authentication 
		try: 
			# create OAuthHandler object 
			self.auth = OAuthHandler(consumer_key, consumer_secret) 
			# set access token and secret 
			self.auth.set_access_token(access_token, access_token_secret) 
			# create tweepy API object to fetch tweets 
			self.api = tweepy.API(self.auth) 
		except: 
			print("Error: Authentication Failed") 

	def clean_tweet(self, tweet): 
		''' 
		Utility function to clean tweet text by removing links, special characters 
		using simple regex statements. 
		'''
		return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 

	def get_tweet_sentiment(self, tweet): 
		''' 
		Utility function to classify sentiment of passed tweet 
		using textblob's sentiment method 
		'''
		# create TextBlob object of passed tweet text 
		analysis = TextBlob(self.clean_tweet(tweet)) 
		# set sentiment 
		if analysis.sentiment.polarity > 0: 
			return 'non toxic'
		elif analysis.sentiment.polarity == 0: 
			return 'neutral'
		else: 
			return 'toxic'

	def get_tweets(self, query, count = 10): 
		''' 
		Main function to fetch tweets and parse them. 
		'''
		# empty list to store parsed tweets 
		tweets = [] 

		try: 
			# call twitter api to fetch tweets 
			fetched_tweets = self.api.search(q = query, count = count) 

			# parsing tweets one by one 
			for tweet in fetched_tweets: 
				# empty dictionary to store required params of a tweet 
				parsed_tweet = {} 

				# saving text of tweet 
				parsed_tweet['text'] = tweet.text 
				# saving sentiment of tweet 
				parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 

				# appending parsed tweet to tweets list 
				if tweet.retweet_count > 0: 
					# if tweet has retweets, ensure that it is appended only once 
					if parsed_tweet not in tweets: 
						tweets.append(parsed_tweet) 
				else: 
					tweets.append(parsed_tweet) 

			# return parsed tweets 
			return tweets 

		except tweepy.TweepError as e: 
			# print error (if any) 
			print("Error : " + str(e)) 

def main(): 
	# creating object of TwitterClient Class 
	api = TwitterClient() 
	# calling function to get tweets 
	tweets = api.get_tweets(query = 'goa', count = 500) 

	# picking positive tweets from tweets 
	ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'non toxic'] 
	# percentage of positive tweets 
	print("Non Toxic tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
	# picking negative tweets from tweets 
	ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'toxic'] 
	# percentage of negative tweets 
	print("Toxic tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
	# percentage of neutral tweets 
	print("Neutral tweets percentage: {} %".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets))) 

	# printing first 3 positive tweets 
	print("\n\nNon Toxic tweets:") 
	for tweet in ptweets[:3]: 
		print(tweet['text']) 

	# printing first 3 negative tweets 
	print("\n\nToxic tweets:") 
	for tweet in ntweets[:3]: 
		print(tweet['text']) 

if __name__ == "__main__": 
	# calling main function 
	main() 



Non Toxic tweets percentage: 31.34328358208955 %
Toxic tweets percentage: 8.955223880597014 %
Neutral tweets percentage: 59.701492537313435 %


Non Toxic tweets:
RT @IOC_Maharashtra: In this @IndianOilcl year of #Digitalisation,  @IOC_Maharashtra e-launched the Sales promotion scheme of #free petrol…
Goa Tourism Appealing Visitors From Across the Globe https://t.co/8qVXemsNlV
@goa_kev @2Malarkey @LKTranslator Oh, the old 'my list of followers is bigger than yours'..... Dear oh dear.


Toxic tweets:
RT @Wright4Kids: Is this for real? This is desperate beyond belief? There is no way this survey would meet the Education Minister’s high st…
RT @prudentgoa: As Goa scores double century,Goemkars ask why unofficially testing stopped Mangor, why 2000 odd tests that CM promised not…
RT @KumarKetkar: If your leader's degree is questionable, then you may not have an idea students’ problems. In Goa, the ABVP demands that e…


### Analysis was performed to find percentage of toxic,non-toxic and neutral tweets on query 'Goa'.